# The Long and Short of Fama-French Momentum

Most investors are long-only and do not have access to investment vehicles that go short. However, academic factor investing focuses mostly on long-short portfolios to show their persistence. The contribution of the short-side of the portfolio to aggregate performance cannot be trivialized. Here, we construct three portfolios out of the Fama-French Momentum data-set -- long-only, short-only and long-short -- to get an idea of how they intersect.

The [Fama-French](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_10_port_form_pr_12_2_daily.html) data-set has returns for portfolios constructed out of each decile of prior returns. With **HI_PRIOR** and **LO_PRIOR** returns, long-only, long-short and short-only portfolio daily returns can be calculated. These returns can then be compared with market returns contained in the [5 Factors (2x3)](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_5_factors_2x3.html) data-set by adding back the **Rf** to **Rm-Rf**.

The documentation for the Fama-French data-set can be found [here](https://plutopy.readthedocs.io/en/latest/FamaFrench.html) and [here](https://shyams80.github.io/plutoR/docs/reference/FamaFrench-class.html)

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")

#initialize
famaFrench <- FamaFrench()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from

In [2]:
hiMom <- famaFrench$MomentumDaily() %>%
    filter(KEY_ID == 'HI_PRIOR') %>%
    select(TIME_STAMP, RET) %>%
    collect() %>%
    as.data.frame()

loMom <- famaFrench$MomentumDaily() %>%
    filter(KEY_ID == 'LO_PRIOR') %>%
    select(TIME_STAMP, RET) %>%
    collect() %>%
    as.data.frame()

mktRet <- famaFrench$FiveFactor3x2Daily() %>%
    inner_join(famaFrench$FiveFactor3x2Daily(), by=c('TIME_STAMP')) %>%
    filter(KEY_ID.x == 'MKT-RF' & KEY_ID.y == 'RF') %>%
    mutate(R = RET.x + RET.y) %>%
    select(TIME_STAMP, R) %>%
    collect() %>%
    as.data.frame()

head(loMom)
head(hiMom)
head(mktRet)

tail(loMom)
tail(hiMom)
tail(mktRet)

TIME_STAMP,RET
<date>,<dbl>
1926-11-03,0.12
1926-11-03,-0.12
1926-11-04,3.54
1926-11-04,0.65
1926-11-05,-2.26
1926-11-05,-0.84


TIME_STAMP,RET
<date>,<dbl>
1926-11-03,0.61
1926-11-03,1.28
1926-11-04,0.21
1926-11-04,0.40
1926-11-05,1.13
1926-11-05,0.08


TIME_STAMP,R
<date>,<dbl>
1963-07-01,-0.658
1963-07-02,0.802
1963-07-03,0.642
1963-07-05,0.412
1963-07-08,-0.618
1963-07-09,0.462


,TIME_STAMP,RET
,<date>,<dbl>
48823,2019-06-26,0.55
48824,2019-06-26,1.69
48825,2019-06-27,1.43
48826,2019-06-27,1.07
48827,2019-06-28,0.95
48828,2019-06-28,1.24


,TIME_STAMP,RET
,<date>,<dbl>
48823,2019-06-26,-0.66
48824,2019-06-26,-0.51
48825,2019-06-27,1.85
48826,2019-06-27,0.47
48827,2019-06-28,1.60
48828,2019-06-28,0.35


,TIME_STAMP,R
,<date>,<dbl>
14090,2019-06-21,-0.201
14091,2019-06-24,-0.331
14092,2019-06-25,-0.971
14093,2019-06-26,-0.051
14094,2019-06-27,0.609
14095,2019-06-28,0.689


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)